In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define Image DataGenerators for data augmentation (optional)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Define training and validation data paths (replace with your paths)
train_dir = "/Users/jowinjosephraju/Miniproject/D:/mini project/Testing videos/Ex-Guard 2/train"
val_dir = "/Users/jowinjosephraju/Miniproject/D:/mini project/Testing videos/Ex-Guard 2/valid"

# Set image dimensions (same as AlexNet input)
img_width, img_height = 224, 224
num_classes=3
# Define AlexNet architecture
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(11, 11), strides=(4, 4), activation="relu", input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(filters=192, kernel_size=(5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(filters=384, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))  # Replace num_classes with your actual number of classes

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs_50alexnet/",histogram_freq=1)


# Load training and validation data using the generators (optional for augmentation)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,  # Adjust batch size based on your hardware
    class_mode="categorical"
)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode="categorical"
)

# Train the model
model.fit(train_generator, epochs=50, validation_data=val_generator,callbacks=[tb_callback])  # Adjust epochs for training time



Found 1939 images belonging to 3 classes.
Found 552 images belonging to 3 classes.
Epoch 1/50


2024-05-04 21:08:44.843209: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


61/61 [==============================] - 42s 577ms/step - loss: 0.9432 - accuracy: 0.5926 - val_loss: 0.7613 - val_accuracy: 0.6431
Epoch 2/50
61/61 [==============================] - 34s 559ms/step - loss: 0.8056 - accuracy: 0.6338 - val_loss: 0.7623 - val_accuracy: 0.6431
Epoch 3/50
61/61 [==============================] - 35s 565ms/step - loss: 0.8029 - accuracy: 0.6349 - val_loss: 0.7664 - val_accuracy: 0.6431
Epoch 4/50
61/61 [==============================] - 35s 573ms/step - loss: 0.8007 - accuracy: 0.6349 - val_loss: 0.7631 - val_accuracy: 0.6431
Epoch 5/50
61/61 [==============================] - 34s 561ms/step - loss: 0.8023 - accuracy: 0.6349 - val_loss: 0.7614 - val_accuracy: 0.6431
Epoch 6/50
61/61 [==============================] - 34s 555ms/step - loss: 0.8010 - accuracy: 0.6349 - val_loss: 0.7674 - val_accuracy: 0.6431
Epoch 7/50
61/61 [==============================] - 34s 555ms/step - loss: 0.8053 - accuracy: 0.6338 - val_loss: 0.7670 - val_accuracy: 0.6431
Epoch 8/50

In [4]:
# Save the trained model (optional)
model.save("alexnet_model.h5")

/Users/jowinjosephraju/Miniproject/tensorenv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
%load_ext tensorboard
%tensorboard --logdir logs_50alexnet/

In [7]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import os
from tensorboardX import SummaryWriter  # Assuming you have TensorBoard installed

# Define hyperparameters
num_classes = 3  # Replace with the number of classes in your dataset
batch_size = 32
learning_rate = 0.001
epochs = 10

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),  # Resize (adjust as needed)
        transforms.RandomResizedCrop(224),  # Randomly crop and resize to 224x224 (AlexNet input size)
        transforms.RandomHorizontalFlip(),  # Random horizontal flip (optional)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize for ImageNet
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),  # Resize (adjust as needed)
        transforms.CenterCrop(224),  # Center crop and resize to 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

# Define custom datasets (replace with your implementation)
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.image_paths = []
        self.labels = []

        # Load image paths and labels from your dataset directory structure (modify as needed)
        for class_name in os.listdir(data_dir):
            class_path = os.path.join(data_dir, class_name)
            for filename in os.listdir(class_path):
                image_path = os.path.join(class_path, filename)
                self.image_paths.append(image_path)
                self.labels.append(your_class_list.index(class_name))  # Assuming class names in directory names

        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)  # Use OpenCV to read the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format (if needed)

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

# Load pre-trained AlexNet model
model = torchvision.models.alexnet(pretrained=True)

# Freeze feature extraction layers (transfer learning)
for param in model.features.parameters():
    param.requires_grad = False

# Modify the final classifier for your num_classes
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=learning_rate)

# Define custom datasets for training and validation (replace with your data paths)
train_data_dir = 
val_data_dir = 

train_dataset = MyDataset(train_data_dir, transform=data_transforms['train'])
val_dataset = MyDataset(val_data_dir, transform=data_transforms['val'])

# Create data loaders
train_data = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data = DataLoader(val_dataset, batch_size=batch_size)

# Create a TensorBoard writer instance (specify a log directory)
writer = SummaryWriter


/Users/jowinjosephraju/Miniproject/tensorenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jowinjosephraju/Miniproject/tensorenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /Users/jowinjosephraju/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
  6%|██████▎                                                                                                            | 12.9M/233M [00:08<02:24, 1.60MB/s

KeyboardInterrupt: 